In [1]:
import sqlite3
import pandas as pd

In [2]:
db = sqlite3.connect('Hop_Teaming_2018.sqlite')

In [3]:
with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    sqliteCursor = db.cursor()
    db.execute('CREATE UNIQUE INDEX IF NOT EXISTS nppes_npi ON nppes (npi)')
    db.execute('CREATE UNIQUE INDEX IF NOT EXISTS taxonomy_code ON taxonomy (code)')
    db.execute('CREATE INDEX IF NOT EXISTS nppes_organization_name ON nppes (`provider_organization_name_(legal_business_name)`)')

## First, build a profile of providers referring patients to the major hospitals in Nashville. Are certain specialties more likely to refer to a particular hospital over the others?

entity_type_code = 2 includes urgent care, doctors offices + minor hospitals (not just major hospitals)

EXPLAIN QUERY PLAN - show estimated query run time 

In [10]:
query = '''
SELECT t.specialization, n2.`provider_organization_name_(legal_business_name)`, 
       SUM(hop.transaction_count) AS patients_referral
FROM Hop_Teaming_2018 AS hop
INNER JOIN nppes n1
ON hop.from_npi = n1.npi
INNER JOIN nppes n2
ON hop.to_npi = n2.npi 
INNER JOIN taxonomy as t
ON n1.primary_taxonomy = t.code
WHERE n1.entity_type_code = 1
    AND n2.provider_business_practice_location_address_city_name = 'NASHVILLE'
    AND n2.provider_business_practice_location_address_state_name = 'TN'
    AND n2.entity_type_code = 2
    AND t.specialization <> 'None'
GROUP BY t.specialization, n2.`provider_organization_name_(legal_business_name)`
ORDER BY patients_referral DESC
LIMIT 20;
'''

In [11]:
with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    Nashville_sqlite = pd.read_sql(query, db)

In [12]:
Nashville_sqlite

,specialization,provider_organization_name_(legal_business_name),patients_referral
0,Anatomic Pathology & Clinical Pathology,"ASSOCIATED PATHOLOGISTS, LLC",358401
1,Anatomic Pathology & Clinical Pathology,ASSOCIATED PATHOLOGISTS LLC,304988
2,Anatomic Pathology & Clinical Pathology,"PATHGROUP LABS, LLC",212407
3,Diagnostic Radiology,VANDERBILT UNIVERSITY MEDICAL CENTER,201804
4,Diagnostic Radiology,RADIOLOGY ALLIANCE PC,201363
5,Cardiovascular Disease,VANDERBILT UNIVERSITY MEDICAL CENTER,190753
6,Hematology & Oncology,TENNESSEE ONCOLOGY PLLC,176556
7,Diagnostic Radiology,SAINT THOMAS MEDICAL PARTNERS,118707
8,Medical Oncology,TENNESSEE ONCOLOGY PLLC,105409
9,Diagnostic Radiology,TENNESSEE ONCOLOGY PLLC,95919
